In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

C:\Users\janga\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
df=pd.read_csv(r"C:\Users\janga\Pubg Task\data.csv")

In [ ]:
df

In [ ]:
df.shape

In [ ]:
df.info()

## Dropping Unnamed:0 column

In [ ]:
df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [ ]:
df.head()

* Here we dropped Unnamed:0 Column from the dataset.Because there is no title for the Column.

* So we can dropped the column

# Checking Missing Values

In [ ]:
df.isnull().sum()

* Here we found only one missing value in winPlacePerc.
* So we can impute with mean,Because of winPlacePerc is continuous data.

In [ ]:
mean=round(np.mean(df.winPlacePerc))
df.winPlacePerc.fillna(mean,inplace=True)

In [ ]:
df.isnull().sum()

***After Filling Data Thier is no loss of Data***

In [ ]:
df.describe()

## EXPLORATORY DATA ANALYSIS

### Checking how many types of versions in Pubg

In [ ]:
match_types = df.loc[:,"matchType"].value_counts().to_frame().reset_index()
match_types.columns = ["Type","Count"]
match_types

* The above frame shows,In Pubg how many types of matches are avaialble.and how many types of matches they played in Pubg with different types.

### Checking how many matches in our data set

In [ ]:
number_of_matches = df.loc[:,"matchId"].nunique()
number_of_matches

* There are 47951 matches in our dataset


In [ ]:
plt.figure(figsize=(18,6))
sns.countplot(x='matchType',data=df)
plt.xticks(rotation=90)
plt.show()

* The graph above shows that the most popular game modes are squad and duo. Next I will aggregate all these individual types into three main categories (squad, duo and solo).

In [ ]:
match_types1 = df.loc[:,"matchType"].value_counts().to_frame()
aggregated_squads = match_types1.loc[["squad-fpp","squad","normal-squad-fpp","normal-squad"],"matchType"].sum()
aggregated_duos = match_types1.loc[["duo-fpp","duo","normal-duo-fpp","normal-duo"],"matchType"].sum()
aggregated_solo = match_types1.loc[["solo-fpp","solo","normal-solo-fpp","normal-solo"],"matchType"].sum()
aggregated_match = pd.DataFrame([aggregated_squads,aggregated_duos,aggregated_solo], index=["squad","duo","solo"], columns =["count"])
aggregated_match

* In PUBG there are essentially three main modes of game: Solo, Duo and Squad.
* So we are aggregate individual types into three main categories

In [ ]:
fig1, ax1 = plt.subplots(figsize=(5, 5))
labels = ['squad', 'duo', 'solo']
wedges, texts, autotexts = ax1.pie(aggregated_match["count"],textprops=dict(color="w"), autopct='%1.1f%%')
ax1.legend(wedges, labels,title="Types",loc="center left",bbox_to_anchor=(1, 0, 0.5, 1))
plt.setp(autotexts, size=12, weight="bold")
plt.show()

* the above pie chart shows that 54% of all the matches were played in squad mode

In [ ]:
plt.figure(figsize=(15,8))
sns.distplot(df['numGroups'])
plt.title('Number of groups')
plt.show()

* The above graph allows to clearly says distribution three spikes referring to squad games, duo games and solo games.

In [ ]:
plt.figure(figsize=(15,8))
sns.boxplot(x='kills',y='damageDealt',data=df)
plt.title('Damage Dealth VS Number of kills')
plt.show()

* There is an obvious correlation between number of kills and damage dealt. We see also that there are some outliers, more in the lower range. As the number of kills increases number of outliers reduces. The maximum kills is 72.

In [ ]:
df[df['kills']>50][["Id","assists","damageDealt","headshotKills","kills","longestKill"]]

In [ ]:
headshots = df[df['headshotKills']>0]
plt.figure(figsize=(15,5))
sns.countplot(headshots['headshotKills'].sort_values())
print("Maximum number of headshots that the player scored: " + str(df["headshotKills"].max()))

In [ ]:
headshots = df[df['DBNOs']>0]
plt.figure(figsize=(15,5))
sns.countplot(headshots['DBNOs'].sort_values())
print("Mean number of DBNOs that the player scored: " + str(df["DBNOs"].mean()))

In [ ]:
sns.barplot(x="revives",y="groupId",data=df)

### Now lets try to find if there is any correlation between kills and DBNOs

* It seems that DBNOs are correlated with kills. That makes sense as usually if player is not killed by headshoot you have to finish him while he's in DBNO state. Interesting is the first observation in the plot - apparently there is a number of players who scored a kill without DBNOs - this is usually a headshot or a vechicle explosion

### MAXIMUM DISTANCE

In [ ]:
dist = df[df['longestKill']<200]
dist.hist('longestKill', bins=20, figsize = (16,8))
plt.show()

In [ ]:
print("Average longest kill distance a player achieve is {:.1f}m, 95% of them not more than {:.1f}m and a maximum distance is {:.1f}m." .format(df['longestKill'].mean(),df['longestKill'].quantile(0.95),df['longestKill'].max()))

In [ ]:
walk = df["walkDistance"] == 0
print('number of players didnot walk at all:',walk.sum())

In [ ]:
ride = df["rideDistance"] == 0
print('number of players who didnot ride at all:',ride.sum())

In [ ]:
swim=df['swimDistance']==0
print('number of players who dint swim at all',swim.sum())

### Now lets create the sum of walking,swimming and driving distances

In [ ]:
travel_dist = df["walkDistance"] + df["rideDistance"] + df["swimDistance"]
travel_dist = travel_dist[travel_dist<5000]
travel_dist.hist(bins=40, figsize = (15,10))
plt.show()

### NOW LETS ANALYSE THE TOP 10 PLAYERS IN THE GIVEN DATASET

In [ ]:
top10 = df[df["winPlacePerc"]>0.9]
print("TOP 10 overview\n")
print("Average number of kills: {:.1f}\nMinimum: {}\nThe best: {}\n95% of players within: {} kills." 
      .format(top10["kills"].mean(), top10["kills"].min(), top10["kills"].max(),top10["kills"].quantile(0.95)))

In [ ]:
print("On average the best 10% of players have the longest kill at {:.3f} meters, and the best score is {:.1f} meters."
      .format(top10["longestKill"].mean(), top10["longestKill"].max()))

### Effect of Killing on Winning Percentage

In [ ]:
kills = df.copy()
kills['killsCategories'] = pd.cut(kills['kills'], [-1, 0, 2, 5, 10, 60], labels=['0_kills','1-2_kills', '3-5_kills', '6-10_kills', '10+_kills'])
plt.figure(figsize=(15,8))
sns.boxplot(x="killsCategories", y="winPlacePerc", data=kills)
plt.show()

### Checking Outliers and Outliers Treatment

In [ ]:
plt.figure(figsize=(19,12))


num_features = df.select_dtypes(include=['int64']).columns
for i in range(6):
    plt.subplot(2,3,i+1)
    plt.boxplot(df[num_features[i]])
    plt.title(num_features[i],color="b",fontsize=20)
    plt.yticks(fontsize=14)
    plt.xticks(fontsize=14)

In [ ]:
from scipy.stats.mstats import winsorize
df["assists"]           = winsorize(df["assists"],(0,0.18))
df["boosts"]        = winsorize(df["boosts"],(0,0.090))
df["DBNOs"]  = winsorize(df["DBNOs"],(0,0.090))
df["headshotKills"]  = winsorize(df["headshotKills"],(0,0.17))
df["heals"]  = winsorize(df["heals"],(0,0.090))
df["killPlace"]= winsorize(df["killPlace"],(0.,0.090))

In [ ]:
plt.figure(figsize=(19,12))


num_features = df.select_dtypes(include=['int64']).columns
for i in range(6):
    plt.subplot(2,3,i+1)
    plt.boxplot(df[num_features[i]])
    plt.title(num_features[i],color="b",fontsize=20)
    plt.yticks(fontsize=14)
    plt.xticks(fontsize=14)

* There are some of the outliers we found in the assists,boosts,DBNOs....etc
* We treated outliers using winsorize method.

In [ ]:
from scipy.stats import norm 
age=np.array(df.kills)

In [ ]:
plt.figure(figsize=(10, 8))
plt.title('kills', size=8)
plt.xlabel('Value in $', size=8)
sns.distplot(kills, fit=norm, kde=False)

In [ ]:
sample_num = 1000
sample_size = 30

mean_sample_values = []

for i in range(sample_num):
    sample_mean = np.mean(np.random.choice(kills, sample_size, replace=True))
    mean_sample_values.append(sample_mean)